In [6]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier
from tpot import TPOTClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [3]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [4]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams(aggregated_data)
# Run script

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import FastICA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

# Score on the training set was:-0.5443811951009836
# 'log loss' pipeline
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

# exported_pipeline = RandomForestClassifier(bootstrap=True, criterion="gini", max_features=0.55, min_samples_leaf=5, min_samples_split=9, n_estimators=100)

# exported_pipeline = DecisionTreeClassifier(criterion="gini", max_depth=7, min_samples_leaf=11, min_samples_split=12)

# exported_pipeline.fit(training_features, training_target)

#exported_pipeline = LogisticRegression()

# 'accuracy' pipeline
exported_pipeline = make_pipeline(
    FastICA(tol=0.0),
    FastICA(tol=0.0),
    Normalizer(norm="l2"),
    GaussianNB()
)

exported_pipeline.fit(np.array(X), np.array(Y))


//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


Pipeline(memory=None,
     steps=[('fastica-1', FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
    n_components=None, random_state=None, tol=0.0, w_init=None,
    whiten=True)), ('fastica-2', FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
    n_components=None, random_state=None, tol=0.0, w_init=None,
    whiten=True)), ('normalizer', Normalizer(copy=True, norm='l2')), ('gaussiannb', GaussianNB(priors=None))])

In [6]:
# Create id to team name mappings
teamNames = pd.read_csv('./ncaa-data/Teams.csv')
def teamIDNameMapping():
    team_id_map = {}
    for index, row in teamNames.iterrows():
        team_id_map[row['TeamID']] = row['TeamName']
    
    return team_id_map

team_id_map = teamIDNameMapping()

In [7]:
def WinChanceMapping(year, exported_pipeline):
    
    submission_data = []
    tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
    tourney_teams = tourney_seeds[tourney_seeds['Season'] == year]['Team']

#     stat_fields = ['score', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF',
#                    'fgp', '3pp', 'ftp']
    stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
    
    tourney_teams.sort_values()
    
    for team_1 in tourney_teams:
        for team_2 in tourney_teams:
            if team_1 != team_2:
                prediction = mm_predictor.predict_winner(
                    team_1, team_2, exported_pipeline, 2017, stat_fields)
                label = '2017_' + str(team_1) + '_' + str(team_2)
                submission_data.append([label, prediction[0][0]])
    
    
    teamTeamWinChanceMap = {}
    for line in submission_data:
        title = line[0].split('_')
        team1 = team_id_map[int(title[1])]
        team2 = team_id_map[int(title[2])]
        winRate = str(round(line[1]* 100,2))
        if team1 not in teamTeamWinChanceMap:
            teamTeamWinChanceMap[team1] = {}
        if float(winRate) >= 50:
            if team1 in teamTeamWinChanceMap and team2 in teamTeamWinChanceMap[team1] and teamTeamWinChanceMap[team1][team2] > float(winRate):
                continue
            if team2 in teamTeamWinChanceMap and team1 in teamTeamWinChanceMap[team2] and teamTeamWinChanceMap[team2][team1] < float(winRate):
                # update teamTeammap[team2][team1] to remove that key
                teamTeamWinChanceMap[team2].pop(team1, None)
            teamTeamWinChanceMap[team1][team2] = float(winRate)
            
    return teamTeamWinChanceMap

In [8]:
# print("Writing %d results." % len(submission_data))
# with open('./ncaa-data/submission.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['id', 'pred'])
#     writer.writerows(submission_data)

chanceMap = WinChanceMapping(2017, exported_pipeline)

In [9]:
def filterTourneyDataByYear(year):

    tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
    tourney_data = tourney_data[tourney_data.Season == year]
    
    return tourney_data

def getWinnersList(tourney_data):
    winners = []

    tourney_data_grouped = tourney_data[4:].groupby('WTeamID').size().reset_index(name='NumWins')
    for index, row in tourney_data_grouped.iterrows():
        teamName = team_id_map[row['WTeamID']]
        wins = row['NumWins']
        if len(winners) == 0:
            winners.append([])
        winners[0].append(teamName)
        if wins > 1:
            if len(winners) == 1:
                winners.append([])
            winners[1].append(teamName)
        if wins > 2:
            if len(winners) == 2:
                winners.append([])
            winners[2].append(teamName)
        if wins > 3:
            if len(winners) == 3:
                winners.append([])
            winners[3].append(teamName)
        if wins > 4:
            if len(winners) == 4:
                winners.append([])
            winners[4].append(teamName)
        if wins > 5:
            if len(winners) == 5:
                winners.append([])
            winners[5].append(teamName)
        if wins > 6:
            if len(winners) == 6:
                winners.append([])
            winners[6].append(teamName)
    
    return winners

In [10]:
# Method that takes in season's tournament data, id to name mapping of teams
def calcBracketScore(teamTeamWinChanceMap, tourney_data):
    
    winners = getWinnersList(tourney_data)
    
    # First four rows is 'first four' and not in actual bracket
    firstFour = tourney_data[:4]
    mainTourney = tourney_data[4:]
    
    tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
    
    tourney = [-1] * max(tourney_seeds['Team'])

    # Look at first four teams
    for index, row in firstFour.iterrows():
        index1 = int(row['WTeamID'])
        index2 = int(row['LTeamID'])
        team1Name = team_id_map[index1]
        team2Name = team_id_map[index2]

        if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]):
            tourney[index2] = index1
        else:
            tourney[index1] = index2

    # MAIN TOURNAMENT 
    score = 0
    for index, row in mainTourney.iterrows():
        index1 = int(row['WTeamID'])
        index2 = int(row['LTeamID'])
        while tourney[index1] > 0:
            index1 = tourney[index1]
        while tourney[index2] > 0:
            index2 = tourney[index2]
        team1Name = team_id_map[index1]
        team2Name = team_id_map[index2]

        if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]): # team1 would win
            tourney[index2] = index1
            tourney[index1] = tourney[index1] - 1
            if team1Name in winners[abs(tourney[index1]) - 2]:
                score += 2**(abs(tourney[index1]) - 2) * 10
#                 print(team1Name + ' vs ' + team2Name + ', team 1 wins')
#                 print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
#             else: # Delete else statemenet when done debugging
#                 print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team1 wins')

        else: #team2 would win
            tourney[index1] = index2
            tourney[index2] = tourney[index2] - 1
            if team2Name in winners[abs(tourney[index2]) - 2]:
                score += 2**(abs(tourney[index2]) - 2) * 10
#                 print(team1Name + ' vs ' + team2Name + ', team 2 wins')
#                 print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
#             else: # Delete else statement when done debugging
#                 print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team2 wins')

    return score

In [11]:
data_2017 = filterTourneyDataByYear(2017)
data_2017
print(calcBracketScore(chanceMap, data_2017))

610


In [17]:
data_2017 = filterTourneyDataByYear(2017)

for i in range(100):
    print(i)
    X,Y = mm_predictor.analyze_teams(aggregated_data)
    
    exported_pipeline1 = make_pipeline(
        StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
        RobustScaler(),
        StackingEstimator(estimator=GaussianNB()),
        StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
        StackingEstimator(estimator=GaussianNB()),
        LogisticRegression(C=5.0, dual=False, penalty="l1")
    )
    
    exported_pipeline1.fit(np.array(X), np.array(Y))
    
    chanceMap = WinChanceMapping(2017, exported_pipeline1)
    
    scores1.append(calcBracketScore(chanceMap, data_2017))
    
    
    exported_pipeline2 = make_pipeline(
        FastICA(tol=0.0),
        FastICA(tol=0.0),
        Normalizer(norm="l2"),
        GaussianNB()
    )

    exported_pipeline2.fit(np.array(X), np.array(Y))
    chanceMap = WinChanceMapping(2017, exported_pipeline2)
    scores2.append(calcBracketScore(chanceMap, data_2017))
    
print(scores1)


0


//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


1


//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


2


...

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


28


//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


29


KeyboardInterrupt: 

In [3]:
print(scores1)

[620, 610, 610, 650, 810, 650, 650, 650, 660, 830, 670, 670, 830, 830, 830, 670, 670, 670, 670, 670, 670, 670, 670, 590, 590, 670, 830, 670, 590, 670, 670, 830, 590, 590, 590, 590, 830, 590, 590, 590, 670, 590, 590]


In [1]:
# From before
scores1 = [620, 610, 610, 650, 810, 650, 650, 650, 660, 830, 670, 670, 830, 830, 830, 670, 670, 670, 670, 670, 670, 670, 670, 590, 590, 670, 830, 670, 590, 670, 670, 830, 590, 590, 590, 590, 830, 590, 590, 590, 670, 590, 590]

In [22]:
np.mean(scores1)

670.93023255813955

In [25]:
pd.Series(scores1).describe()

count     43.000000
mean     670.930233
std       82.658620
min      590.000000
25%      590.000000
50%      670.000000
75%      670.000000
max      830.000000
dtype: float64

In [24]:
print(scores2)

[680, 700, 640, 920, 780, 750, 940, 920, 930, 620, 750, 890, 930, 960, 940, 690, 730, 710, 790, 740, 780, 760, 940, 900, 830, 920, 950, 650, 790, 930, 780, 770, 700, 600, 920, 790, 780, 750, 770, 800, 940, 940, 930]


In [4]:
# From before
scores2 = [680, 700, 640, 920, 780, 750, 940, 920, 930, 620, 750, 890, 930, 960, 940, 690, 730, 710, 790, 740, 780, 760, 940, 900, 830, 920, 950, 650, 790, 930, 780, 770, 700, 600, 920, 790, 780, 750, 770, 800, 940, 940, 930]

In [23]:
np.mean(scores2)

812.32558139534888

In [7]:
pd.Series(scores2).describe()

count     43.000000
mean     812.325581
std      106.923141
min      600.000000
25%      745.000000
50%      790.000000
75%      925.000000
max      960.000000
dtype: float64

In [ ]:
# def bracketScorer(y_true, y_predicted):
#     # y_predicted must be a chance map... we don't care what y_true is rofl
#     bracket = filterTourneyDataByYear(2017)
#     chanceMap = WinChanceMapping(2017)
#     return calcBracketScore(chanceMap, bracket)

In [ ]:
# from sklearn.metrics import make_scorer

# bballScorer = make_scorer(bracketScorer, greater_is_better=True)

In [ ]:
# tppot = TPOTClassifier(generations = 1, max_time_mins=780, verbosity=2, scoring = bballScorer, population_size=10)

# tppot.fit(np.array(X), np.array(Y))